In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, SimpleRNN, LSTM, GRU, Dense

In [2]:
num_words= 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

In [3]:
maxlen = 100
x_train = pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=maxlen, padding='post', truncating='post')

In [4]:
version_fn = getattr(tf.keras, "version", None)
if version_fn and version_fn().startswith("3."):
    import tensorflow.keras as keras
else:
    keras = tf.keras

In [31]:
from tensorflow.keras.layers import Input
embedding =100
encoder_input = Input(shape=(None,))
model = Sequential()
model.add(encoder_input)
model.add(Embedding(input_dim=num_words, output_dim=embedding, input_length=maxlen))
model.add(Bidirectional(LSTM(5)))
model.add(Dense(1, activation='sigmoid'))
model.build()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=num_words,
                    epochs=1)
model.summary()


c:\Users\HP\Desktop\VSCode\Bidirection_lstm\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 813ms/step - accuracy: 0.5117 - loss: 0.6924


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

In [6]:
train_loss, train_acc = model.evaluate(x_test, y_test, verbose=2)

782/782 - 7s - 9ms/step - accuracy: 0.5677 - loss: 0.6911


In [7]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [8]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [9]:
log_dir

'logs/fit20240724-225541'

In [10]:
model_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [11]:
import mlflow
earLy_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [12]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, callbacks=[model_callbacks, earLy_stop]
)

Epoch 1/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 45s 57ms/step - accuracy: 0.7009 - loss: 0.5623 - val_accuracy: 0.8047 - val_loss: 0.4251
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 43s 55ms/step - accuracy: 0.8715 - loss: 0.3155 - val_accuracy: 0.8017 - val_loss: 0.4543
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 52s 67ms/step - accuracy: 0.9199 - loss: 0.2140 - val_accuracy: 0.7948 - val_loss: 0.5056
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 52s 67ms/step - accuracy: 0.9512 - loss: 0.1428 - val_accuracy: 0.7976 - val_loss: 0.5553
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 52s 67ms/step - accuracy: 0.9719 - loss: 0.0940 - val_accuracy: 0.7953 - val_loss: 0.6803
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 45s 58ms/step - accuracy: 0.9792 - loss: 0.0683 - val_accuracy: 0.7878 - val_loss: 0.7406
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 50s 64ms/step - accuracy: 0.9876 - loss: 0.0426 - val_accuracy: 0.7861 - val_loss: 0.8290
Epoch 8/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 53ms/step - accuracy: 0.9936 - loss: 0

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

In [14]:
from mlflow.models.signature import infer_signature
model_sign = infer_signature(x_test, model.predict(x_test))

782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step


In [15]:
exp_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
exp_name= "IMDB_bidirection_lstm_time:" + exp_timestamp
#exp_name = "IMDB_bidirection_lstm"
run_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_name = "IMDB_bidirection_lstm_run:" + run_timestamp

In [16]:
def tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign):
    mlflow.set_experiment(exp_name)
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("batch_size", num_words )
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_param("epochs", 100)
        mlflow.log_metric("train_loss", train_loss)
        mlflow.log_metric("train_accuracy", train_acc)
        mlflow.tensorflow.log_model(model, "IMDB", signature=model_sign)
    mlflow.end_run()

In [17]:
tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign)

2024/07/24 23:04:18 INFO mlflow.tracking.fluent: Experiment with name 'IMDB_bidirection_lstm_time:20240724-230418' does not exist. Creating a new experiment.


In [27]:
model.export(filepath='saved_model', format='tf')
model.save('models\BI_LSTM_model.h5')

<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:2: SyntaxWarning: invalid escape sequence '\B'
C:\Users\HP\AppData\Local\Temp\ipykernel_20392\1228302281.py:2: SyntaxWarning: invalid escape sequence '\B'
  model.save('models\BI_LSTM_model.h5')


INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, None), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2189617793168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2189617794128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2189617794320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2189617793744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2189617794512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2189617794704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2189617795280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2189617792208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2189617794896: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [29]:
from tensorflow.keras.models import load_model # type: ignore
import bentoml # type: ignore
from tensorflow.keras.layers import TFSMLayer # type: ignore

model1 = load_model('models\BI_LSTM_model.h5' )
bentoml.tensorflow.save_model('bento_model', model1)
print(f"{model1.tag}")

<>:5: SyntaxWarning: invalid escape sequence '\B'
<>:5: SyntaxWarning: invalid escape sequence '\B'
C:\Users\HP\AppData\Local\Temp\ipykernel_20392\3116275401.py:5: SyntaxWarning: invalid escape sequence '\B'
  model1 = load_model('models\BI_LSTM_model.h5' )
C:\Users\HP\AppData\Local\Temp\ipykernel_20392\3116275401.py:5: SyntaxWarning: invalid escape sequence '\B'
  model1 = load_model('models\BI_LSTM_model.h5' )


ValueError: The layer sequential has never been called and thus has no defined input.